In [83]:
import random
import re
from collections import defaultdict

In [84]:
def generar_tabla_frecuencias_condicionales(archivo, N=1):
    """
    Genera una tabla de frecuencias condicionales de palabras a partir de un archivo de texto.
    :param archivo: Nombre del archivo de texto en formato UTF-8.
    :param N: Número de palabras condicionantes.
    :return: Diccionario con frecuencias condicionales.
    """
    with open(archivo, 'r', encoding='utf-8') as file:
        texto = file.read().lower()
    
    palabras = re.findall(r'\b\w+\b', texto) # quito puntuación y divido en palabras
    
    # Diccionario para almacenar las frecs condicionales
    tabla_frecuencias = defaultdict(lambda: defaultdict(int))       #Nota: Cada vez que se accede a una clave que no existe en este diccionario interno, se inicializa con 0

    # Si se accede a una clave que no existe en tabla_frecuencias, se crea automáticamente un nuevo defaultdict(int).
    # Si se accede a una clave que no existe en ese defaultdict(int), se crea un entero con valor 0.

    #Tabla de frecuencias
    for i in range(len(palabras) - N):           # crea secuencias de palabras de longitud N (condiciones) y cunta la frecuencia con la que aparecen otras palabras justo después de esas condiciones.
        condicion = tuple(palabras[i:i + N])  # N palabras como condición
        siguiente_palabra = palabras[i + N]
        tabla_frecuencias[condicion][siguiente_palabra] += 1

    return tabla_frecuencias


In [85]:
def guardar_tabla_frecuencias_en_archivo(tabla_frecuencias, nombre_archivo="Tablas_Frecuencias.txt"):
    """
    Guarda la tabla de frecuencias condicionales en un archivo de texto.
    :param tabla_frecuencias: Diccionario con frecuencias condicionales.
    :param nombre_archivo: Nombre del archivo donde se guardarán las frecuencias.
    """
    with open(nombre_archivo, 'w', encoding='utf-8') as file:

        for condicion, frecuencias in tabla_frecuencias.items():
            condicion_str = ' '.join(condicion)                     # Convertir la tupla de palabras en una cadena de texto
            file.write(f"Condición: {condicion_str}\n")

            for palabra_siguiente, frecuencia in frecuencias.items():
                file.write(f"  {palabra_siguiente}: {frecuencia}\n")

            file.write("\n")  # Espacio entre diferentes condiciones

    print(f"Tabla de frecuencias guardada en {nombre_archivo}")

In [86]:
def generar_texto_markov(tabla_frecuencias, longitud_texto, N=1):
    """
    Genera un texto aleatorio basado en Markov.
    :param tabla_frecuencias: tabla de frecuencias condicionales
    :param longitud_texto: longitud del texto generado
    :param N: número de palabras previas
    :return: texto generado
    """
    palabras_previas_estado = random.choice(list(tabla_frecuencias.keys()))     # Ver
    texto_generado = list(palabras_previas_estado)

    for _ in range(longitud_texto - N):
        if palabras_previas_estado not in tabla_frecuencias:                # Si no hay palabras previas, cambio
            palabras_previas_estado = random.choice(list(tabla_frecuencias.keys()))  

        siguiente_palabra = random.choices(
            list(tabla_frecuencias[palabras_previas_estado].keys()), 
            list(tabla_frecuencias[palabras_previas_estado].values())
        )[0]
        
        texto_generado.append(siguiente_palabra)            # Ver
        palabras_previas_estado = tuple(texto_generado[-N:])  # Actualiza el palabras previas para las próximas palabras
    
    return ' '.join(texto_generado)


In [87]:
def guardar_texto_en_archivo(texto, nombre_archivo='Fuente_Markov.txt'):
    """
    Guarda el texto generado en un archivo.
    :param texto: el texto generado
    :param nombre_archivo: nombre del archivo de texto donde se guardará
    """
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo:
        archivo.write(texto)
    print(f"Texto generado guardado en {nombre_archivo}")

In [88]:
archivo_texto = "La_Divina_Comedia.txt"
N = 3  # cadena de palabras para analizar (condicion)

In [89]:
# Generar la tabla de frecuencias condicionales

tabla_frecuencias = generar_tabla_frecuencias_condicionales(archivo_texto, N)
guardar_tabla_frecuencias_en_archivo(tabla_frecuencias)

Tabla de frecuencias guardada en Tablas_Frecuencias.txt


In [90]:
# Generar texto aleatorio basado en Markov

longitud_texto = 100
texto_generado = generar_texto_markov(tabla_frecuencias, longitud_texto, N)

In [91]:
guardar_texto_en_archivo(texto_generado)

Texto generado guardado en Fuente_Markov.txt


In [92]:
def utf8_a_binario(texto):
    """
    Convierte una cadena de texto a su representación binaria usando codificación UTF-8.
    """
    return ''.join(format(byte, '08b') for byte in texto.encode('utf-8'))

In [93]:
def codigo_unario_binario(longitud):        # El código unario es '1' repetido (longitud - 1) veces seguido de '0'.
    """
    Codifica la longitud usando el código unario-binario.
    """
    if longitud == 1:
        return '0'  # Caso especial, la longitud 1 se representa con solo un '0'
    else:
        unario = '1' * (longitud - 1) + '0'
        return unario

In [94]:
def lz77_comprimir(texto, ventana_tamano=8, buffer_tamano=32):      
    """
    Implementa la compresión LZ77 y genera una cadena de bits (y un metodo visual de comprobacion).
    :param texto: cadena de caracteres de entrada
    :param ventana_tamano: tamaño de la ventana de búsqueda (en este caso prubo 8)
    :param buffer_tamano: tamaño del buffer de previsualización
    :return: cadena de unos y ceros (binaria) representando la compresión
    """
    i = 0
    resultado_binario = []  # Guarda el resultado comprimido en binario
    resultado_visual = []   # Guarda la visualización del proceso de compresión
    
    while i < len(texto):

        mejor_longitud = 0
        mejor_desplazamiento = 0
        ventana_inicio = max(0, i - ventana_tamano)  # La ventana se mueve con un tamaño de 8
        
       
        # Busca la mejor "coincidencia" en la ventana
        for j in range(ventana_inicio, i):

            longitud = 0

            while (longitud < buffer_tamano and i + longitud < len(texto) and
                   texto[j + longitud] == texto[i + longitud]):
                longitud += 1

            if longitud > mejor_longitud:
                mejor_longitud = longitud
                mejor_desplazamiento = i - j



        # Codifica la coincidencia o literal
        if mejor_longitud >= 3:

            # Coincidencia
            desplazamiento_bin = format(mejor_desplazamiento, '03b')  # 3 bits para el desplazamiento en la ventana
            longitud_bin = codigo_unario_binario(mejor_longitud)
            resultado_binario.append(longitud_bin + desplazamiento_bin)
            
            # Comprobación visual
            resultado_visual.append(f"Coincidencia: '{texto[i:i+mejor_longitud]}'(longitud={mejor_longitud}, offset={mejor_desplazamiento}) -> Binario: {longitud_bin} {desplazamiento_bin}")
            
            i += mejor_longitud
        
        else:
            # Codificar literal en UTF-8
            literal_bin = utf8_a_binario(texto[i])
            resultado_binario.append(literal_bin)
            
            # Comprobación visual
            resultado_visual.append(f"Literal: '{texto[i]}' -> Binario: {literal_bin}")
            
            i += 1

    for linea in resultado_visual:  # Muestrta la comprobación visual del proceso
        print(linea)

    return ''.join(resultado_binario)

In [95]:
def lz77_descomprimir(binario, ventana_tamano=8):
    """
    Implementa la descompresión LZ77 a partir de una cadena de bits.
    :param binario: cadena binaria comprimida con LZ77
    :param ventana_tamano: tamaño de la ventana de búsqueda (en este caso pruebo 8)
    :return: cadena de caracteres descomprimida
    """
    i = 0
    resultado = []
    
    while i < len(binario):

        if binario[i] == '0':  # Esto indica el caso de un literal

            literal_binario = binario[i:i + 8]
            literal = chr(int(literal_binario, 2))
            resultado.append(literal)
            i += 8

        else:  # Esto indica una coincidencia de bits

            longitud = 1

            while binario[i] == '1':
                longitud += 1
                i += 1
            i += 1          # Ver - Soluciona error - salta el '0' que marca el final de la parte unaria
            desplazamiento = int(binario[i:i + 3], 2)
            i += 3

            pos_inicio = len(resultado) - desplazamiento # # Agregar la "coincidencia" al resultado
            
            for _ in range(longitud):
                resultado.append(resultado[pos_inicio])
                pos_inicio += 1
    
    return ''.join(resultado)

In [96]:
texto_prueba = 'abcabcabcabcabcabc'
binario_comprimido = lz77_comprimir(texto_prueba)
texto_descomprimido = lz77_descomprimir(binario_comprimido)

print("\nTexto original:", texto_prueba)
print("Texto descomprimido:", texto_descomprimido)

Literal: 'a' -> Binario: 01100001
Literal: 'b' -> Binario: 01100010
Literal: 'c' -> Binario: 01100011
Coincidencia: 'abcabcabcabcabc'(longitud=15, offset=3) -> Binario: 111111111111110 011

Texto original: abcabcabcabcabcabc
Texto descomprimido: abcabcabcabcabcabc
